In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import logging
from datetime import datetime, timezone

from app.database_redis.connection import get_redis_client
from app.services.apis.streamqueue_service.client import StreamQueueServiceAPI
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings

logger = logging.getLogger(__name__)

check_and_process_connections_interval_sec: 5.0


In [3]:
from app.services.audio.redis import Diarisation,Transcript

In [4]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [5]:
#await redis_client.flushdb()

In [18]:
meeting_id = 'https://meet.google.com/tdi-fqhq-icr'
diarization = Diarisation(meeting_id,redis_client=redis_client)
transcript = Transcript(meeting_id,redis_client=redis_client)

In [19]:
#await diarization.delete()

In [20]:
#await transcript.delete()

In [21]:
import json

In [22]:
diarizations = []
while True:
    d = await diarization.rpop()
    if not d:
        break
    diarizations.append(json.loads(d))

In [23]:
diarizations

[[[{'start': 7.122241086587437,
    'end': 15.169779286926996,
    'speaker': '6a3902ac-d4c3-4409-b3f9-d6a6d26439fd',
    'score': 0.8141123}],
  '2024-05-22T20:21:25.064897+00:00'],
 [[{'start': 0.008488964346349746,
    'end': 1.1120543293718166,
    'speaker': '6a3902ac-d4c3-4409-b3f9-d6a6d26439fd',
    'score': 0.9200109},
   {'start': 1.2648556876061121,
    'end': 8.616298811544992,
    'speaker': '6a3902ac-d4c3-4409-b3f9-d6a6d26439fd',
    'score': 0.9200109},
   {'start': 8.803056027164686,
    'end': 14.151103565365027,
    'speaker': '6a3902ac-d4c3-4409-b3f9-d6a6d26439fd',
    'score': 0.9200109},
   {'start': 14.660441426146011,
    'end': 15.203735144312395,
    'speaker': '6a3902ac-d4c3-4409-b3f9-d6a6d26439fd',
    'score': 0.9200109},
   {'start': 16.29032258064516,
    'end': 21.99490662139219,
    'speaker': '6a3902ac-d4c3-4409-b3f9-d6a6d26439fd',
    'score': 0.9200109},
   {'start': 22.011884550084893,
    'end': 23.132427843803057,
    'speaker': '6a3902ac-d4c3-4409-

In [28]:
import pandas as pd

/tmp/ipykernel_458/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [13]:
import pickle

In [29]:
#TODO @Andrew this is to apply to the list of transcriptions for concatination
def transcrtiption2df(transcription,seek):
    df =pd.DataFrame(transcription[0])
    df['seek'] = pd.to_datetime(seek)
    df['conv_start'] = pd.to_timedelta(df['start'],unit='s')+df['seek']
    df['conv_end'] = pd.to_timedelta(df['end'],unit='s')+df['seek']
    return df.reset_index(drop=True)

def concatinate_segments(segments):
    dfs = [transcrtiption2df(result,seek) for result, seek in segments]
    df = pd.concat(dfs).sort_values(by=['conv_start', 'conv_end']).reset_index(drop=True)

    # Initialize an empty list to store the final words
    final_words = []

    # Iterate over the concatenated dataframe and select words with the highest probability
    for i in range(len(df)):
        if i == 0:
            final_words.append(df.loc[i])
        else:
            if df.loc[i]['conv_start'] >= final_words[-1]['conv_end']:
                final_words.append(df.loc[i])
            else:
                if df.loc[i]['probability'] > final_words[-1]['probability']:
                    final_words[-1] = df.loc[i]

    # Convert the list of final words back to a dataframe
    return pd.DataFrame(final_words)

In [305]:
with open('diarizations.pkl', 'wb') as file:
    pickle.dump(diarizations, file)

In [26]:
with open('transcriptions.pkl', 'wb') as file:
    pickle.dump(transcriptions, file)

In [24]:
transcriptions = []

In [144]:
while True:
    d = await transcript.rpop()
    if not d:
        break
    transcriptions.append(json.loads(d))
concatinate_segments(transcriptions)
''.join(concatinate_segments(transcriptions)['word'].tolist()).split('.')

[" to be able to we can with your code inside of GitHub, I with you the fail when it tries to edit the my now i have a demo to share we're gonna hope the demo goes smoothly task kind of my favorite uh you know hello world task right a bash script that prints You can see as it's going about its work, You can see as it's going about its work, pops up I want to add to this",
 ' can do this kind of wrote brand new where, I know, it achieves some task and you can say, okay, now I have some extra requirement that I want to add to this',
 " The most exciting thing, can I really to show you all is, with the what I'm going to do actually show you all what I've done here",
 " an empty staging area, an empty workspace, I'll actually show you all what I've done here",
 ' Out of this to basically to, to do is tell OpenDevon to work inside of this codebase, basically Open Devon codebase',
 ' again',
 " I should be able to see So you can see now we're inside of the Open Devon codebase",
 " This is th

In [181]:
t = await transcript.rpop();t
json.loads(t)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [39]:
settings.check_and_process_connections_interval_sec

5.0

In [4]:


class Processor:
    def __init__(self):
        self.__running_tasks = set()
        self.stream_queue_service_api = StreamQueueServiceAPI()

    async def process_connections(self):
        logger.info("Process connections...")
        connections = await self.stream_queue_service_api.get_connections()
        connection_ids = [c[0] for c in connections]

        for connection_id in connection_ids:
            await self._process_connection_task(connection_id)

    async def _process_connection_task(self, connection_id, diarizer_step=60, transcriber_step=5):
        redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)
        meeting_id, segment_start_timestamp, segment_end_timestamp, user_id = await self.writestream2file(connection_id)

        current_time = datetime.now(timezone.utc)

        connection = Connection(redis_client, connection_id, user_id)
        await connection.update_timestamps(segment_start_timestamp, segment_end_timestamp)

        meeting = Meeting(redis_client, meeting_id)
        await meeting.load_from_redis()
        await meeting.add_connection(connection.id)
        meeting.diarizer_last_updated_timestamp = meeting.diarizer_last_updated_timestamp or segment_start_timestamp
        meeting.transcriber_last_updated_timestamp = meeting.transcriber_last_updated_timestamp or segment_start_timestamp

        if (current_time - meeting.diarizer_last_updated_timestamp).seconds > diarizer_step:
            diarizer = Diarizer(redis_client)
            await diarizer.add_todo(meeting.meeting_id)
            await meeting.update_diarizer_timestamp(
                segment_start_timestamp, diarizer_last_updated_timestamp=current_time
            )

        if (current_time - meeting.transcriber_last_updated_timestamp).seconds > transcriber_step:
            transcriber = Transcriber(redis_client)
            await transcriber.add_todo(meeting.meeting_id)
            await meeting.update_transcriber_timestamp(
                segment_start_timestamp, transcriber_last_updated_timestamp=current_time
            )

    async def writestream2file(self, connection_id):
        path = f"/audio/{connection_id}.webm"
        first_timestamp = None
        items = await self.stream_queue_service_api.fetch_chunks(connection_id, num_chunks=100)

        if items:
            # if there is no meeting_id in META-data
            meeting_id = connection_id

            for item in items["chunks"]:
                chunk = bytes.fromhex(item["chunk"])
                first_timestamp = datetime.fromisoformat(item["timestamp"].rstrip('Z')).astimezone(timezone.utc) if not first_timestamp else first_timestamp

                # Open the file in append mode
                with open(path, "ab") as file:
                    # Write data to the file
                    file.write(chunk)

                last_timestamp = datetime.fromisoformat(item["timestamp"].rstrip('Z')).astimezone(timezone.utc)

                meeting_id = item["meeting_id"]
                user_id = item["user_id"]

            return meeting_id, first_timestamp, last_timestamp, user_id


In [7]:
diarizer_step=10
transcriber_step=5

In [8]:
self = Processor()

In [9]:
connections = await self.stream_queue_service_api.get_connections()
connection_ids = [c[0] for c in connections]

In [10]:
connection_id = connection_ids[0]

In [11]:
connection_id

'25b0a1b4-78e7-4bf3-9d1c-c64bf59b845d'

In [23]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)
meeting_id, segment_start_timestamp, segment_end_timestamp, user_id = await self.writestream2file(connection_id)

In [36]:

current_time = datetime.now(timezone.utc)

connection = Connection(redis_client, connection_id, user_id)
await connection.update_timestamps(segment_start_timestamp, segment_end_timestamp)

meeting = Meeting(redis_client, meeting_id)
await meeting.load_from_redis()

In [37]:
await meeting.set_start_timestamp(segment_start_timestamp)

In [38]:

await meeting.add_connection(connection.id)

meeting.diarizer_last_updated_timestamp = meeting.diarizer_last_updated_timestamp or segment_start_timestamp
meeting.transcriber_last_updated_timestamp = meeting.transcriber_last_updated_timestamp or segment_start_timestamp

if (current_time - meeting.diarizer_last_updated_timestamp).seconds > diarizer_step:
    diarizer = Diarizer(redis_client)
    await diarizer.add_todo(meeting.meeting_id)
    await meeting.update_diarizer_timestamp(
        segment_start_timestamp, diarizer_last_updated_timestamp=current_time
    )
    print("diarizer added")

if (current_time - meeting.transcriber_last_updated_timestamp).seconds > transcriber_step:
    transcriber = Transcriber(redis_client)
    await transcriber.add_todo(meeting.meeting_id)
    await meeting.update_transcriber_timestamp(
        segment_start_timestamp, transcriber_last_updated_timestamp=current_time
    )
    print("transcriber added")

diarizer added
transcriber added


In [29]:
meeting.diarizer_last_updated_timestamp

datetime.datetime(2024, 5, 22, 15, 41, 22, 776157, tzinfo=tzutc())

In [30]:
current_time

datetime.datetime(2024, 5, 22, 15, 41, 22, 776157, tzinfo=datetime.timezone.utc)

In [21]:
connection_id = connection_ids[0]

In [35]:
connection_id

'87a3f4b1-5b50-45be-b2c7-dfd83ebfe022'

In [36]:
meeting_id, segment_start_timestamp, segment_end_timestamp, user_id = await self.writestream2file(connection_id)

In [37]:
segment_start_timestamp,segment_end_timestamp

(datetime.datetime(2024, 5, 22, 14, 27, 59, 846448, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 5, 22, 14, 28, 9, 48882, tzinfo=datetime.timezone.utc))

In [38]:
meeting_id

'https://meet.google.com/jmt-yfgt-rzp'

In [39]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [40]:
#await redis_client.flushdb()

In [41]:
current_time = datetime.now(timezone.utc)

connection = Connection(redis_client, connection_id, user_id)
await connection.update_timestamps(segment_start_timestamp, segment_end_timestamp)

In [42]:
#transcriber_step

In [43]:
current_time

datetime.datetime(2024, 5, 22, 14, 28, 11, 963135, tzinfo=datetime.timezone.utc)

In [52]:


meeting = Meeting(redis_client, meeting_id)

await meeting.update_redis()
await meeting.load_from_redis()
await meeting.add_connection(connection.id)
meeting.diarizer_last_updated_timestamp = meeting.diarizer_last_updated_timestamp or segment_start_timestamp
meeting.transcriber_last_updated_timestamp = meeting.transcriber_last_updated_timestamp or segment_start_timestamp

In [53]:
meeting.diarizer_last_updated_timestamp

datetime.datetime(2024, 5, 22, 14, 27, 59, 846448, tzinfo=tzutc())

In [54]:
diarizer_step

60

In [55]:
meeting.diarizer_last_updated_timestamp

datetime.datetime(2024, 5, 22, 14, 27, 59, 846448, tzinfo=tzutc())

In [64]:

if (current_time - meeting.diarizer_last_updated_timestamp).seconds > diarizer_step:
    print("diarizer added")
    diarizer = Diarizer(redis_client)
    await diarizer.add_todo(meeting.meeting_id)
    await meeting.update_diarizer_timestamp(
        segment_start_timestamp, diarizer_last_updated_timestamp=current_time
    )

if (current_time - meeting.transcriber_last_updated_timestamp).seconds > transcriber_step:
    print("transcriber added")
    transcriber = Transcriber(redis_client)
    await transcriber.add_todo(meeting.meeting_id)
    await meeting.update_transcriber_timestamp(
        segment_start_timestamp, transcriber_last_updated_timestamp=current_time
            )

In [65]:
settings

Settings(service_version='0.0.1_example', service_name='Audio API', service_api_host='0.0.0.0', service_api_port=8009, service_token='service_token', check_and_process_connections_interval_sec=5.0, stream_queue_service_list_connections='http://host.docker.internal:8000/api/v1/connections/list', stream_queue_service_flush_cache='http://host.docker.internal:8000/api/v1/tools/flush-cache', stream_queue_service_get_next_chunks='http://host.docker.internal:8000/api/v1/tools/get-next-chunks', stream_queue_service_health='http://host.docker.internal:8000/api/v1/health', stream_queue_service_health_check='http://host.docker.internal:8000/api/v1/hc', stream_queue_service_request_timeout=5, stream_queue_service_auth_token='LKJBn98wefgh', redis_host='redis', redis_port=6379, redis_password='', volume_data_path='/home/dima/ssd/0', redis_image_port='6382')

In [66]:
(current_time - meeting.transcriber_last_updated_timestamp).seconds

0

In [67]:
meeting.transcriber_last_updated_timestamp

datetime.datetime(2024, 5, 22, 14, 28, 11, 963135, tzinfo=tzutc())

In [68]:
meeting.transcriber_last_updated_timestamp

datetime.datetime(2024, 5, 22, 14, 28, 11, 963135, tzinfo=tzutc())

In [69]:
segment_start_timestamp

datetime.datetime(2024, 5, 22, 14, 27, 59, 846448, tzinfo=datetime.timezone.utc)

In [70]:
connection.start_timestamp

datetime.datetime(2024, 5, 22, 14, 27, 59, 846448, tzinfo=datetime.timezone.utc)

In [71]:
meeting.transcriber_last_updated_timestamp

datetime.datetime(2024, 5, 22, 14, 28, 11, 963135, tzinfo=tzutc())